In [ ]:
import pandas as pd
import numpy as np
from itertools import product

from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

In [ ]:
def train_xgboost_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)

    model = xgb.XGBRegressor(objective="reg:squarederror")
    model.fit(X_train_flat.astype(float), y_train.astype(float))
    predictions = model.predict(X_test_flat.astype(float))
    mse = mean_squared_error(y_test.astype(float), predictions)
    mae = mean_absolute_error(y_test.astype(float), predictions)
    r2 = r2_score(y_test.astype(float), predictions)
    mape = mean_absolute_percentage_error(y_test.astype(float), predictions)

    return {
        "y_test": y_test,
        "predictions": predictions,
        "mse": mse,
        "mae": mae,
        "r2": r2,
        "mape": mape,
    }

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_data=pd.read_csv("/content/drive/MyDrive/data_timeseries_minmaxscaler.csv")

In [ ]:
df_data.head()

,Unnamed: 0,subject_id,hadm_id,charttime,50882,50912,50971,50983,51006,51222,51265,51301
0,0,10000032,22595853.0,['2180-05-07 05:05:00'],[0.2],[0.004054054054054054],[0.140625],[0.5871559633027523],[0.08196721311475409],[0.5450643776824035],[0.024774774774774775],[0.004781965159968121]
1,1,10000032,22841357.0,['2180-06-27 05:10:00'],[0.17692307692307693],[0.004054054054054054],[0.16796875],[0.48623853211009194],[0.09508196721311475],[0.5321888412017167],[0.04954954954954955],[0.007514516679949903]
2,2,10000032,25742920.0,"['2180-08-06 06:36:00', '2180-08-07 06:15:00']","[0.18461538461538463, 0.16923076923076924]","[0.008108108108108109, 0.005405405405405406]","[0.2109375, 0.17578125]","[0.48623853211009194, 0.45871559633027537]","[0.12131147540983607, 0.09508196721311475]","[0.51931330472103, 0.49785407725321884]","[0.04804804804804805, 0.038288288288288286]","[0.008539223499943073, 0.00637595354662416]"
3,3,10000032,29079034.0,"['2180-07-24 06:35:00', '2180-07-25 04:45:00']","[0.16923076923076924, 0.19230769230769232]","[0.005405405405405406, 0.005405405405405406]","[0.16015625, 0.16796875]","[0.5229357798165137, 0.5229357798165137]","[0.09180327868852459, 0.11475409836065574]","[0.5107296137339056, 0.4806866952789699]","[0.03340840840840841, 0.033783783783783786]","[0.004668108846635546, 0.005465103039963566]"
4,4,10000084,23052089.0,"['2160-11-20 22:30:00', '2160-11-21 07:00:00',...","[0.13076923076923078, 0.16923076923076924, 0.1...","[0.010810810810810811, 0.00945945945945946, 0....","[0.19140625, 0.12109375, 0.125, 0.125, 0.12109...","[0.5412844036697249, 0.559633027522936, 0.5871...","[0.05573770491803279, 0.036065573770491806, 0....","[0.5493562231759657, 0.5708154506437768, 0.596...","[0.09684684684684684, 0.10548048048048048, 0.1...","[0.009791642946601389, 0.009222361379938518, 0..."


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
def preprocess_df(df, scaler=MinMaxScaler(), columns_to_scale=['Bic', 'Crt', 'Pot', 'Sod', 'Ure', 'Hgb', 'Plt', 'Wbc'], num_bins=10, pivot=True):
    if pivot:
        mrl_sorted = df.sort_values(by=['subject_id', 'hadm_id', 'chartday', 'itemid', 'charthour'])
        mrl_sampled = mrl_sorted.groupby(['subject_id', 'hadm_id', 'chartday', 'itemid']).first().reset_index()
        mrl_full = mrl_sampled.pivot(index=['subject_id', 'hadm_id', 'chartday'], columns='itemid', values='valuenum').reset_index()
        mrl = mrl_full.dropna()
    else:
        mrl = df.dropna()

    # replace the id if is contained in the columns
    ids = {50882: 'Bic', 50912: 'Crt', 50971: 'Pot', 50983: 'Sod', 51006: 'Ure', 51222: 'Hgb', 51265: 'Plt', 51301: 'Wbc'}
    for column in mrl.columns:
        for id in ids.keys():
            if str(id) in str(column):
                mrl = mrl.rename(columns={column: str(column).replace(str(id), ids[id])})
                break

    #columns_to_scale = [col for col in mrl.columns for id in ids.values() if id in col]
    #mrl = mrl.rename(columns=ids)
    #columns_to_scale = ['Bic', 'Crt', 'Pot', 'Sod', 'Ure', 'Hgb', 'Plt', 'Wbc']

    # Scale selected columns
    mrl[columns_to_scale] = scaler.fit_transform(mrl[columns_to_scale])

    return mrl

In [ ]:
df=preprocess_df(df_data)

In [ ]:
df.head()

itemid,subject_id,hadm_id,chartday,Bic,Crt,Pot,Sod,Ure,Hgb,Plt,Wbc
0,10000032,22595853.0,2180-05-07,0.200000,0.004054,0.143969,0.571429,0.081967,0.577273,0.025404,0.004782
1,10000032,22841357.0,2180-06-27,0.176923,0.004054,0.171206,0.473214,0.095082,0.563636,0.050808,0.007515
2,10000032,25742920.0,2180-08-06,0.184615,0.008108,0.214008,0.473214,0.121311,0.550000,0.049269,0.008539
3,10000032,25742920.0,2180-08-07,0.176923,0.005405,0.175097,0.410714,0.091803,0.527273,0.039261,0.006376
5,10000032,29079034.0,2180-07-24,0.169231,0.005405,0.163424,0.508929,0.091803,0.540909,0.034257,0.004668


In [ ]:
#preprocess
ids = {'50882': 'Bic', '50912': 'Crt', '50971': 'Pot', '50983': 'Sod', '51006': 'Ure', '51222': 'Hgb', '51265': 'Plt', '51301': 'Wbc'}
df=df_data.rename(columns=ids)
df=df.dropna()

In [ ]:
#get matrix for each row in dataframe
columns_to_scale = ['Bic', 'Crt', 'Pot', 'Sod', 'Ure', 'Hgb', 'Plt', 'Wbc']
df['matrix']=df[columns_to_scale].apply(lambda row: [row[col].strip('][').split(', ') for col in columns_to_scale],axis=1)

In [ ]:
df['matrix'].head()

0    [[0.2], [0.004054054054054054], [0.140625], [0...
1    [[0.17692307692307693], [0.004054054054054054]...
2    [[0.18461538461538463, 0.16923076923076924], [...
3    [[0.16923076923076924, 0.19230769230769232], [...
4    [[0.13076923076923078, 0.16923076923076924, 0....
Name: matrix, dtype: object

In [ ]:
def get_dfs(data):
  dataframes=[]
  for matrix in data:
    df=pd.DataFrame(matrix).transpose()
    df.columns=['Bic', 'Crt', 'Pot', 'Sod', 'Ure', 'Hgb', 'Plt', 'Wbc']
    dataframes.append(df)
  return dataframes

In [ ]:
dataframes=get_dfs(df['matrix'].values)

In [ ]:
import pickle
with open('/content/drive/MyDrive/per_patient_dfs.pkl', 'wb') as f:
    pickle.dump(dataframes, f)

In [ ]:
ex=pd.DataFrame(df['matrix'][10]).transpose()

In [ ]:
ex.columns=['Bic', 'Crt', 'Pot', 'Sod', 'Ure', 'Hgb', 'Plt', 'Wbc']

In [ ]:
ex

,Bic,Crt,Pot,Sod,Ure,Hgb,Plt,Wbc
0,0.14615384615384616,0.02027027027027027,0.12890625,0.6422018348623855,0.10819672131147541,0.6137339055793992,0.06156156156156156,0.011841056586587727
1,0.17692307692307693,0.01891891891891892,0.11328125,0.63302752293578,0.09836065573770492,0.5450643776824035,0.0487987987987988,0.0084253671866105
2,0.16923076923076924,0.02027027027027027,0.11328125,0.6146788990825689,0.10163934426229508,0.51931330472103,0.04241741741741742,0.005123534099965844
3,0.18461538461538463,0.021621621621621623,0.1171875,0.63302752293578,0.10491803278688525,0.5236051502145922,0.04316816816816817,0.00421268359330525
4,0.18461538461538463,0.021621621621621623,0.1328125,0.63302752293578,0.09836065573770492,0.5364806866952789,0.0472972972972973,0.004781965159968121


In [ ]:
df['matrix'][10]

[['0.14615384615384616',
  '0.17692307692307693',
  '0.16923076923076924',
  '0.18461538461538463',
  '0.18461538461538463'],
 ['0.02027027027027027',
  '0.01891891891891892',
  '0.02027027027027027',
  '0.021621621621621623',
  '0.021621621621621623'],
 ['0.12890625', '0.11328125', '0.11328125', '0.1171875', '0.1328125'],
 ['0.6422018348623855',
  '0.63302752293578',
  '0.6146788990825689',
  '0.63302752293578',
  '0.63302752293578'],
 ['0.10819672131147541',
  '0.09836065573770492',
  '0.10163934426229508',
  '0.10491803278688525',
  '0.09836065573770492'],
 ['0.6137339055793992',
  '0.5450643776824035',
  '0.51931330472103',
  '0.5236051502145922',
  '0.5364806866952789'],
 ['0.06156156156156156',
  '0.0487987987987988',
  '0.04241741741741742',
  '0.04316816816816817',
  '0.0472972972972973'],
 ['0.011841056586587727',
  '0.0084253671866105',
  '0.005123534099965844',
  '0.00421268359330525',
  '0.004781965159968121']]

In [ ]:
def preprocess_data(data,max_length):
  '''
  Preprocess the data by padding or truncating the matrices to max_length
  '''
  X = []
  Y=[]

  # Process each matrix
  for matrix in data:
      matrix=np.array(matrix)
      if matrix.shape[1]<4:
        continue
      Y.append(matrix[:,-1]) #take last time point of matrix as y for model
      matrix=matrix[:,:len(matrix[0])-1] #take all but last time point of matrix as x for model
      num_cols = matrix.shape[1]
      if num_cols < max_length:
          # Pad with zeros
          pad_width = ((0, 0), (0, max_length - num_cols))
          processed_matrix = np.pad(matrix, pad_width, mode='constant', constant_values=0)
      else:
          processed_matrix = matrix[:, :max_length]
      X.append(processed_matrix[:,:max_length-1])


  return np.array(X),np.array(Y)

In [ ]:
X,y=preprocess_data(df['matrix'].values,4)

In [ ]:
np.array(df['matrix'][10]).shape

(8, 5)

In [ ]:
X[0]

array([['0.13076923076923078', '0.16923076923076924',
        '0.16153846153846155'],
       ['0.010810810810810811', '0.00945945945945946',
        '0.00945945945945946'],
       ['0.19140625', '0.12109375', '0.125'],
       ['0.5412844036697249', '0.559633027522936', '0.5871559633027523'],
       ['0.05573770491803279', '0.036065573770491806',
        '0.029508196721311476'],
       ['0.5493562231759657', '0.5708154506437768', '0.5965665236051503'],
       ['0.09684684684684684', '0.10548048048048048',
        '0.10848348348348348'],
       ['0.009791642946601389', '0.009222361379938518',
        '0.009108505066605944']], dtype='<U22')

In [ ]:
y[0]

array(['0.16923076923076924', '0.00945945945945946', '0.12109375',
       '0.6146788990825689', '0.03278688524590164', '0.5536480686695279',
       '0.10472972972972973', '0.007969941933280201'], dtype='<U22')

In [ ]:
# Train the model and get the results
results = train_xgboost_model(X, y)

# You can now print the results or further analyze them
print("Mean Squared Error:", results["mse"])
print("Mean Absolute Error:", results["mae"])
print("R-squared:", results["r2"])
print("Mean Absolute Percentage Error:",results["mape"])

Mean Squared Error: 0.0008476530512804938
Mean Absolute Error: 0.017428945668423918
R-squared: 0.5018349664915689
Mean Absolute Percentage Error: 5281941367.956817


In [ ]:
!pip install hyperimpute

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 33.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 32.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11

In [ ]:
from sklearn.datasets import load_iris
from hyperimpute.plugins.imputers import Imputers
from hyperimpute.utils.benchmarks import compare_models

#X, y = load_iris(as_frame=True, return_X_y=True)



In [ ]:
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [ ]:
columns_to_scale = ['Bic', 'Crt', 'Pot', 'Sod', 'Ure', 'Hgb', 'Plt', 'Wbc']

In [ ]:
dataframes[600]

,Bic,Crt,Pot,Sod,Ure,Hgb,Plt,Wbc
0,0.15384615384615385,0.008108108108108109,0.08984375,0.6146788990825689,0.029508196721311476,0.3776824034334764,0.0731981981981982,0.020266423773198226
1,0.13076923076923078,0.00945945945945946,0.08203125,0.6055045871559634,0.029508196721311476,0.38626609442060084,0.0777027027027027,0.019469429579870207
2,0.14615384615384616,0.010810810810810811,0.0859375,0.5779816513761469,0.04590163934426229,0.2918454935622318,0.06006006006006006,0.019697142206535356
3,0.13076923076923078,0.008108108108108109,0.1015625,0.6055045871559634,0.04590163934426229,0.3690987124463519,0.05555555555555555,0.023112831606512584
4,0.13846153846153847,0.008108108108108109,0.1171875,0.5779816513761469,0.05245901639344262,0.3261802575107296,0.04391891891891892,0.01616759649322555
5,0.14615384615384616,0.00945945945945946,0.1171875,0.6055045871559634,0.05245901639344262,0.33905579399141633,0.05067567567567568,0.014915177046567233
6,0.14615384615384616,0.008108108108108109,0.10546875,0.5871559633027523,0.04918032786885246,0.3605150214592275,0.05780780780780781,0.01400432653990664
7,0.15384615384615385,0.00945945945945946,0.11328125,0.5871559633027523,0.05573770491803279,0.424892703862661,0.0731981981981982,0.016964590686553573
8,0.16153846153846155,0.008108108108108109,0.1015625,0.5871559633027523,0.05245901639344262,0.3648068669527897,0.06794294294294294,0.009905499259933963
9,0.16153846153846155,0.008108108108108109,0.10546875,0.5871559633027523,0.06229508196721312,0.4377682403433476,0.07207207207207207,0.011157918706592282


In [ ]:
!git clone https://github.com/ChenweWu/impute-ehr.git
%cd impute-ehr

Cloning into 'impute-ehr'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 117 (delta 43), reused 110 (delta 36), pack-reused 0
Receiving objects: 100% (117/117), 28.13 MiB | 18.67 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/impute-ehr


In [ ]:
from importlib import reload
import hyperimpute.utils.benchmarks
reload(hyperimpute.utils.benchmarks)
from hyperimpute.utils.benchmarks import compare_models

In [ ]:
d=dataframes[10]

In [ ]:
d[0:4]

,Bic,Crt,Pot,Sod,Ure,Hgb,Plt,Wbc
0,0.14615384615384616,0.02027027027027027,0.12890625,0.6422018348623855,0.10819672131147541,0.6137339055793992,0.06156156156156156,0.011841056586587727
1,0.17692307692307693,0.01891891891891892,0.11328125,0.63302752293578,0.09836065573770492,0.5450643776824035,0.0487987987987988,0.0084253671866105
2,0.16923076923076924,0.02027027027027027,0.11328125,0.6146788990825689,0.10163934426229508,0.51931330472103,0.04241741741741742,0.005123534099965844
3,0.18461538461538463,0.021621621621621623,0.1171875,0.63302752293578,0.10491803278688525,0.5236051502145922,0.04316816816816817,0.00421268359330525


In [ ]:
imputer = Imputers().get("hyperimpute")

results=compare_models(
    name="example",
    evaluated_model=imputer,
    X_raw=dataframes[10],
    ref_methods=["mean",'median','most_frequent','mice'],
    scenarios=["MAR"],
    miss_pct=[0.3],
    n_iter=3,
)

RMSE score


,Scenario,"miss_pct [0, 1]",Evaluated: hyperimpute,mean,median,most_frequent,mice
0,MAR,0.3,0.5108 +/- 0.2345,0.4618 +/- 0.1502,0.4725 +/- 0.1725,0.505 +/- 0.213,0.5214 +/- 0.2957




Wasserstein score


,Scenario,"miss_pct [0, 1]",Evaluated: hyperimpute,mean,median,most_frequent,mice
0,MAR,0.3,0.3799 +/- 0.1896,0.4319 +/- 0.1211,0.4476 +/- 0.1625,0.4475 +/- 0.194,0.3786 +/- 0.1237


In [ ]:
results['rmse'][0][2:]

[(0.5108278175992461, 0.2345109561301142),
 (0.46175540913831653, 0.15021383438258523),
 (0.472510687772319, 0.17246424483760114),
 (0.5049813452481516, 0.213010537569854),
 (0.5214238342238355, 0.29569666655653376)]

In [ ]:
results['rmse'][1][2:]

[(0.6055701787485053, 0.0826747264644927),
 (0.5432445783329819, 0.11182268140844547),
 (0.6440515898416188, 0.12645601073617513),
 (0.5917064613709854, 0.06346059446710049),
 (0.6017305017743151, 0.16010003545582308),
 (0.5584062715342619, 0.08818635561975441),
 (0.7693667503074391, 0.007248297840904832)]

In [ ]:
import random
random.shuffle(dataframes)

In [ ]:
subset=dataframes[:500]

In [ ]:
imputer = Imputers().get("hyperimpute")
rmses=[]
for df in subset:
  if len(df)<4:
    continue
  else:
    df=df[:4]
  try:
    results=compare_models(
      name="example",
      evaluated_model=imputer,
      X_raw=df,
      ref_methods=["mean","median","most_frequent","mice"],
      scenarios=["MAR"],
      miss_pct=[0.3],
      n_iter=3,
    )
    rmses.append(results['rmse'][0][2:])
  except:
    continue

RMSE score


,Scenario,"miss_pct [0, 1]",Evaluated: hyperimpute,mean,median,most_frequent,mice
0,MAR,0.3,0.2915 +/- 0.0403,0.5481 +/- 0.01,0.495 +/- 0.036,0.3889 +/- 0.1753,0.3833 +/- 0.1406




Wasserstein score


,Scenario,"miss_pct [0, 1]",Evaluated: hyperimpute,mean,median,most_frequent,mice
0,MAR,0.3,0.2089 +/- 0.069,0.4631 +/- 0.0854,0.3822 +/- 0.0887,0.2358 +/- 0.1063,0.2657 +/- 0.0947


RMSE score


...

,Scenario,"miss_pct [0, 1]",Evaluated: hyperimpute,mean,median,most_frequent,mice
0,MAR,0.3,0.3784 +/- 0.2345,0.4099 +/- 0.2172,0.3413 +/- 0.2219,0.5158 +/- 0.208,0.6112 +/- 0.2806


RMSE score


,Scenario,"miss_pct [0, 1]",Evaluated: hyperimpute,mean,median,most_frequent,mice
0,MAR,0.3,0.5415 +/- 0.118,0.495 +/- 0.1109,0.5112 +/- 0.125,0.5817 +/- 0.2227,0.6954 +/- 0.1388




Wasserstein score


,Scenario,"miss_pct [0, 1]",Evaluated: hyperimpute,mean,median,most_frequent,mice
0,MAR,0.3,0.4044 +/- 0.0922,0.3529 +/- 0.0587,0.3655 +/- 0.0739,0.4549 +/- 0.1784,0.4129 +/- 0.0569


In [ ]:
rmses

[[(0.29150878727182084, 0.040298453578176255),
  (0.5481044792749761, 0.010008433518139704),
  (0.49495714305892524, 0.03597232882643336),
  (0.38891007187147353, 0.17528974689809895),
  (0.3833251964477302, 0.14060893394555224)],
 [(0.5861384090823587, 0.09342318026838399),
  (0.5834176228894207, 0.11491592307058232),
  (0.6138343730655157, 0.10594898203144798),
  (0.6594867172428137, 0.18010431382999226),
  (0.566207256860953, 0.2229619082857824)],
 [(0.46202153698266146, 0.05263546432766103),
  (0.38893206665125135, 0.15393230213282025),
  (0.4147215407834366, 0.11818991453029792),
  (0.6310547762071739, 0.5113319924753795),
  (0.4776181367117406, 0.37504836103643435)],
 [(0.5901941192853978, 0.07616695337870882),
  (0.48935175613130927, 0.09276457018982004),
  (0.5062014724358171, 0.07395860807143516),
  (0.605559455286901, 0.11621495148473052),
  (0.8459414877640198, 0.2552518026012699)],
 [(0.6120392497020838, 0.011887158029902983),
  (0.48816637838747917, 0.04233258968151772),
 

In [ ]:
len(rmses)

211

In [ ]:
nested_array = np.array(rmses)
# Get the first elements of each tuple
first_elements = nested_array[:,:,0]
means = np.mean(first_elements, axis=0)
rmseResults=dict(zip(['hyperimpute','mean','median','most_frequent','mice'],means))
rmseResults

{'hyperimpute': 0.5943004433502218,
 'mean': 0.5421703167454117,
 'median': 0.5587581898681697,
 'most_frequent': 0.6388780570380637,
 'mice': 0.7154622006297405}